In [ ]:
ENV_TYPE = "TEST"

if(ENV_TYPE != "TEST"):
  !git clone "https://github.com/kk-digital/kcg-ml-sd1p4.git"
  %cd kcg-ml-sd1p4
  !pip3 install -r requirements.txt
  exit()
  base_directory = "./"
else:
  base_directory = "../"

# Magical check for fixing all of our directory issues
import subprocess
output = subprocess.check_output(["pwd"], universal_newlines=True)
if "notebooks" in output:
    %cd ..
del output

In [ ]:
!python3 ./download_models.py

In [ ]:
!python3 ./process_models.py

In [ ]:
import os
import sys

base_directory = "./"
sys.path.insert(0, base_directory)
print(os.path.abspath(base_directory))

import json
import torch
import configparser
import safetensors
from stable_diffusion import StableDiffusion
from stable_diffusion.utils_backend import *
from stable_diffusion.utils_image import *
from stable_diffusion.utils_model import *
from utility.utils_logger import logger

from stable_diffusion.model_paths import *
from configs.model_config import ModelPathConfig

In [ ]:
DEVICE = get_device()
base_dir = os.getcwd()
sys.path.insert(0, base_dir)

In [ ]:
model_config = ModelPathConfig()
pt = IODirectoryTree(model_config)
pt

Here we'll build the `StableDiffusion` class by loading the submodels from disk.

In [ ]:
from transformers import CLIPTextConfig, CLIPTextModel
from stable_diffusion.model.clip_text_embedder import CLIPTextEmbedder
# tokenizer = CLIPTokenizer.from_pretrained('openai/clip-vit-large-patch14')

Instantiate a text embedder:

In [ ]:
text_embedder = CLIPTextEmbedder(device=DEVICE)

Load it's submodels weights:

In [ ]:
text_embedder.load_submodels()

In [ ]:
from stable_diffusion.model.vae import Autoencoder

In [ ]:
autoencoder = Autoencoder(device=DEVICE)

In [ ]:
autoencoder.load(autoencoder_path = VAE_PATH)

In [ ]:
autoencoder.load_submodels()

In [ ]:
from stable_diffusion.model.unet import UNetModel
# from stable_diffusion.utils.model import initialize_unet

In [ ]:
unet_model = UNetModel(device=DEVICE)

In [ ]:
unet_model.load(unet_path = UNET_PATH)

In [ ]:
get_memory_status(DEVICE)

Now we need to build a model with the same structure that the checkpoint we are going to use (by default, `runwayml/stable-diffusion-v1-5`), so the weights get properly mapped. This model is called `LatentDiffusion`. We also have a `initialize_latent_diffusion` function, which I will omit since it's a bit longer than the others.

In [ ]:
from stable_diffusion import LatentDiffusion
# from stable_diffusion.utils.model import initialize_latent_diffusion

In [ ]:
latent_diffusion = LatentDiffusion(
                            autoencoder=autoencoder,
                            clip_embedder=text_embedder,
                            unet_model=unet_model,
                            device=DEVICE
                            )

In [ ]:
latent_diffusion.load(latent_diffusion_path=LATENT_DIFFUSION_PATH)

In [ ]:
from stable_diffusion import StableDiffusion

In [ ]:
stable_diffusion = StableDiffusion(device=DEVICE, model = latent_diffusion, ddim_steps = 20)

In [ ]:
prompt = 'A cat'

In [ ]:
with section('sampling...'):
    image_tensor = stable_diffusion.generate_images(prompt = prompt, seed = 1, negative_prompt = "ugly")

In [ ]:
to_pil(image_tensor.squeeze())

# Nice one!